# Creating a Baseline Tag Labeler

Here we will use [XGBoost](https://xgboost.readthedocs.io/en/latest/python/python_api.html) and [scikit-learn](https://scikit-learn.org/stable/) to create a baseline multi-class, multi-label classifier that will label our sample of [Stack Overflow](http://stackoverflow.com) posts (questions and their answers), two thirds of which lack labels. This will serve as a basis of comparison for the deep network we will train therafter. We will create separate [embeddings](https://keras.io/layers/embeddings/) of their language and code and use these as the signal for our model.

In [1]:
import cupy as cp
import json
import numpy as np
import pandas as pd
import re

print("I'm working!")

I'm working!


#### Load our sample of questions/answers with at least 1 vote and 1 answer

In [2]:
sorted_all_tags = json.load(open('data/stackoverflow/08-05-2019/sorted_all_tags.50000.json'))
max_index = sorted_all_tags[-1][0] + 1

In [3]:
import pyarrow
posts_df = pd.read_parquet(
    'data/stackoverflow/08-05-2019/Questions.Stratified.Final.50000.parquet',
    columns=['_Body'] + ['label_{}'.format(i) for i in range(0, max_index)],
    engine='pyarrow'
)
posts_df.head(5)

,_Body,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,...,label_14,label_15,label_16,label_17,label_18,label_19,label_20,label_21,label_22,label_23
0,"[C, Mono, Winforms, MessageBox, problem, I, fi...",1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[Are, NET, data, providers, Oracle, require, O...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[How, I, focus, foreign, window, I, applicatio...",1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[Default, button, hit, windows, forms, trying,...",1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[Can, I, avoid, JIT, net, Say, code, always, g...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
print(
    '{:,} answered questions with at least 1 upvote'.format(len(posts_df.index))
)

1,293,018 answered questions with at least 1 upvote


## Map from Tags to IDs

In [5]:
tag_index = json.load(open('data/stackoverflow/08-05-2019/tag_index.50000.json'))
index_tag = json.load(open('data/stackoverflow/08-05-2019/index_tag.50000.json'))

## Count the Most Common Tags

In [6]:
label_counts = json.load(open('data/stackoverflow/08-05-2019/label_counts.50000.json'))

# Sanity check the difference files
assert(len(label_counts.keys()) == len(tag_index.keys()) == len(index_tag.keys()) == len(sorted_all_tags))

## To Be Consistent: Make Record Count a Multiple of the Batch Size and Post Sequence Length

Although it is not necessary in our baseline labeler, the Elmo embedding in the network model requires that the number of records be a multiple of the batch size times the number of tokens in the padded posts. We do the same thing to keep the data consistent.

In [7]:
import math

BATCH_SIZE = 32
MAX_LEN = 100
EMBED_SIZE = 50

# Convert label columns to numpy array
labels = posts_df[list(posts_df.columns)[1:]].to_numpy()

# training_count must be a multiple of the BATCH_SIZE times the MAX_LEN for the Elmo embedding layer
highest_factor = math.floor(len(posts_df.index) / (BATCH_SIZE * MAX_LEN))
training_count = highest_factor * BATCH_SIZE * MAX_LEN
print('Highest Factor: {:,} Training Count: {:,}'.format(highest_factor, training_count))

# Remove stopwords - now done in Spark, so can remove once that runs
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

documents = []
for body in posts_df[0:training_count]['_Body'].values.tolist():
    words = body.tolist()
    documents.append(words)

labels = labels[0:training_count]

# Lengths for x and y match
assert( len(documents) == training_count == labels.shape[0])

Highest Factor: 404 Training Count: 1,292,800


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Sample the data to speed development

In [8]:
# import random
# random.seed(33)

# SAMPLE_SIZE = 10000
# id_list = list(range(0, len(filtered_code_words)))
# idx = random.sample(id_list, SAMPLE_SIZE)

# # idx = np.random.choice(np.arange(len(matrix_posts)), SAMPLE_SIZE, replace=False)

# sampled_posts = [x for i, x in enumerate(filtered_code_words) if i in idx]
# sampled_labels = [x for i, x in enumerate(new_labels) if i in idx]

# del filtered_code_words
# del new_labels

# len(sampled_posts), len(sampled_labels)

#### REMINDER: When we add text words we must combine the two valid label lists and then create a new list of labels

In [9]:
# MIN_TEXT = 20

# def extract_text(x):
#     doc = BeautifulSoup(x)
#     codes = doc.find_all('code')
#     [code.extract() if code else None for code in codes]
#     return doc.text

# post_text = tag_posts._Body.apply(extract_text)
# post_text_words = [x.split() for x in post_text.tolist()]

# # Take words with > MIN_TEXT (20) instances
# post_text_words = [[y for y in x if tag_counts[y] > MIN_TEXT] for x in post_text_words]

# # Create a new list of labels to match the new non-empty lists of words
# text_post_ids = defaultdict(bool)
# text_post_id_list = []
# for i, post in enumerate(post_text_words):
#     if len(post) == 0:
#         pass
#     else:
#         text_post_ids[i] = True
#         text_post_id_list.append(i)

#### Encode the tags, replacing their string form with their respective IDs

In [10]:
# encoded_tags = []
# raw_tags = []
# for tagset in coded_tags:
#    encoded_tags.append([1 if id in tagset else 0 for id in id_to_tag.keys()])

# labels = np.array(encoded_tags)

# encoded_tags[0]

## Create a Baseline Gradient Boosted Decision Tree Model

It is useful to have a decision tree model to use as a baseline for comparison with our deep network model. XGBoost's implementation of gradient boosted decision trees is state of the art for this kind of application, but it can't do multi-class, multi-label classification. Therefore we use an [`xgboost.XGBClassifier`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier) with an [`sklearn.multiclass.OneVsRestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) on top of the XGBoost classifier to train one classifier model per label and then apply them to each label to compute the output for each.

We define `VOCAB_SIZE`, `MAX_LENGTH` and `TEST_SPLIT` to define the number of unique words as input into our embedding, the sequence length for each input, and the test/train split for our performance testing.

#### Encode the data using Gensim and Word2Vec

For the network, we'll create our own embeddings. For the baseline model we'll use Word2Vec.

In [11]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer

from gensim.sklearn_api import W2VTransformer
from gensim.models import Word2Vec

import xgboost as xgb

VOCAB_SIZE = 5000
MAX_LENGTH = 100
EMBEDDING_SIZE = 50
NUM_CORES = 12

TEST_SPLIT = 0.2

from os import path

w2v_model  = None
model_path = "data/stackoverflow/08-05-2019/word2vec.50000.model"

# Load the Word2Vec model if it exists
if path.exists(model_path):
    w2v_model = Word2Vec.load(model_path)
else:
    w2v_model = Word2Vec(
        documents,
        size=EMBEDDING_SIZE,
        min_count=1,
        window=10,
        workers=NUM_CORES,
        iter=10,
        seed=33
    )
    w2v_model.save(model_path)

print('Word2Vec model built!')

Word2Vec model built!


In [12]:
w2v_model.wv.most_similar(positive='program')

[('program,', 0.8130234479904175),
 ('programm', 0.7869037985801697),
 ('programme', 0.77663254737854),
 ('script', 0.7494806051254272),
 ('process', 0.7310086488723755),
 ('programs', 0.705868661403656),
 ('code', 0.6764416694641113),
 ('routine', 0.6757915019989014),
 ('computer', 0.6742575168609619),
 ('program.', 0.6669228076934814)]

In [13]:
encoded_docs = [[w2v_model.wv[word] for word in post] for post in documents]
len(encoded_docs)

1292800

In [14]:
len(encoded_docs), len(encoded_docs[0]), encoded_docs[0][0]

(1292800,
 100,
 array([ 5.487012  , -5.1054535 , -1.8988078 , -7.685227  ,  4.207665  ,
        -1.1219728 ,  1.1265227 , -0.25528577, -9.583929  , -1.1784164 ,
        -6.200084  ,  2.2063937 ,  0.5968637 ,  0.12365432, -4.4148273 ,
         8.716778  ,  6.9193225 ,  2.5406606 , -3.1150146 ,  3.4230134 ,
         5.548197  , -6.9893117 ,  1.3929974 , -3.8196619 ,  3.193098  ,
         8.028727  ,  6.06637   ,  6.2713885 , 12.862909  , -0.11642915,
         1.8495387 ,  6.1364207 , -4.3824267 , -2.5337682 ,  2.9778636 ,
         5.652717  ,  6.90736   , -0.9474404 , -1.989368  , -2.7301998 ,
        -4.755108  , -0.313335  ,  2.453273  ,  6.1209016 ,  0.86407304,
         1.9723939 , -3.104216  , -5.3178406 ,  4.0534744 ,  2.2914498 ],
       dtype=float32))

### Pad and limit the posts to MAX_LENGTH (100) words using the average of all words in the corpus

We will now compute a position-wise maximum and minimum, concatenate these values, and use them to pad any documents with less than 20 words. We will simultaneously truncate any documents with more than 20 words. If we were creating our own embeddings using keras we would use [`keras.preprocessing.pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences), but using [`gensim.models.word2vec`](https://radimrehurek.com/gensim/models/word2vec.html) we pad them on our own.

See [Representation learning for very short texts using weighted word embedding aggregation](https://arxiv.org/pdf/1607.00570.pdf) referenced from [Stack Overflow](https://datascience.stackexchange.com/a/17348/59975).

In [15]:
from math import ceil

padded_posts = []
for post in encoded_docs:
    # Pad short posts with alternating min/max
    if len(post) < MAX_LENGTH:
        pointwise_min = np.minimum.reduce(post)
        pointwise_max = np.maximum.reduce(post)
        padding = [pointwise_max, pointwise_min]
        
        post += padding * ceil((MAX_LENGTH - len(post) / 2.0))
        
    # Shorten long posts or those odd number length posts we padded to 51
    if len(post) > MAX_LENGTH:
        post = post[:MAX_LENGTH]
      
    padded_posts.append(post)

# Verify their lengths
assert(min([len(post) for post in padded_posts]) == MAX_LENGTH)
assert(max([len(post) for post in padded_posts]) == MAX_LENGTH)

# Free up the RAM, since we copied the data
# del encoded_docs
len(padded_posts), len(padded_posts[0]), padded_posts[0][0]

(1292800,
 100,
 array([ 5.487012  , -5.1054535 , -1.8988078 , -7.685227  ,  4.207665  ,
        -1.1219728 ,  1.1265227 , -0.25528577, -9.583929  , -1.1784164 ,
        -6.200084  ,  2.2063937 ,  0.5968637 ,  0.12365432, -4.4148273 ,
         8.716778  ,  6.9193225 ,  2.5406606 , -3.1150146 ,  3.4230134 ,
         5.548197  , -6.9893117 ,  1.3929974 , -3.8196619 ,  3.193098  ,
         8.028727  ,  6.06637   ,  6.2713885 , 12.862909  , -0.11642915,
         1.8495387 ,  6.1364207 , -4.3824267 , -2.5337682 ,  2.9778636 ,
         5.652717  ,  6.90736   , -0.9474404 , -1.989368  , -2.7301998 ,
        -4.755108  , -0.313335  ,  2.453273  ,  6.1209016 ,  0.86407304,
         1.9723939 , -3.104216  , -5.3178406 ,  4.0534744 ,  2.2914498 ],
       dtype=float32))

#### Convert the 3D feature array into a wider 2D array

The classifier requires 2D data, so we need to convert our 3D feature array into a wider 2D feature array. We will do this by iterating through the 50 padded elements of Word2Vec vectors for each post and appending them to a long list for each post.

Note that the type of `padded_posts` is `list(list(np.array))`, an artifact of the Word2Vec mapping.

#### Create one Row per Label Column

Training a `sklearn.multiclass.OneVsRestClassifier` with one `xgboost.XGBClassifier` per label exceeded 64GB of RAM and so we are remapping the data to have one instance of the row for each label column in a given row.

For example:

```python
# Input
rows, labels = [0.1, 0.3, 0.4, ...],[0,1,0,1]

# Output
rows_w_labels = [
    ([0.1, 0.3, 0.4, ...], 0),
    ([0.1, 0.3, 0.4, ...], 1),
    ([0.1, 0.3, 0.4, ...], 0),
    ([0.1, 0.3, 0.4, ...], 1)
]
```

In [16]:
import cupy as cp
import sys

# cupy memory management
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()

row_length = MAX_LENGTH * EMBEDDING_SIZE

matrix_posts = []
flat_labels = []
print_shape = False
print_dot = True

print(f'{len(padded_posts):,} padded_posts, {labels.shape} labels')

cpu_posts = []
five_k_count = 0
for i, post in enumerate(padded_posts):
    # Starting with an empty array and append the entire list of embedded words to it, 
    # expanding it's shape to (5000,)
    post = cp.array(post)
    if print_shape and i % 5000 == 0:
        print('post.shape', post.shape)
    
    post_row = cp.concatenate(post, axis=0)
    if print_shape and i % 5000 == 0:
        print('post_row.shape', post_row.shape)
    
    assert(post_row.shape == (row_length,))
    
    # Now add a downward dimension to the data, expanding its dimensions to (1,5000)
    post_row = cp.expand_dims(post_row, axis=0)
    if print_shape and i % 5000 == 0:
        print('post_row.shape', post_row.shape)
    
    assert(post_row.shape == (1,row_length))
    
    if print_shape and i % 5000 == 0:
        print(len(post_row[0]))
    
    matrix_posts.append(post_row)
    
    if print_dot and i % 5000 == 0:
        sys.stderr.write('.')
    
    # Every 25 * 5000 rows, send GPU RAM to main RAM or it will crash with 7618GB cards (g3.4xlarge)
    if i % 5000 == 0:
        five_k_count += 1
        first_five_k = True
        
    if five_k_count % 25 == 0 and first_five_k is True:
        print(f'\ni:{i:,} 5k:{five_k_count:,} GPU RAM --> CPU RAM')
        cpu_posts.append(
            cp.asnumpy(cp.concatenate(matrix_posts, axis=0))
        )
        del matrix_posts
        matrix_posts = []
        mempool.free_all_blocks()
        pinned_mempool.free_all_blocks()
        print('Completed!')
        
        first_five_k = False

# Do the final concat for the remainder over 5K and clear GPU RAM
cpu_posts.append(
    cp.asnumpy(cp.concatenate(matrix_posts, axis=0))
)
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()

# Memory conservation is critical
# del padded_posts
del matrix_posts
len(cpu_posts)

1,292,800 padded_posts, (1292800, 24) labels


.........................


i:120,000 5k:25 GPU RAM --> CPU RAM
Completed!


.........................


i:245,000 5k:50 GPU RAM --> CPU RAM
Completed!


...


i:1,120,000 5k:225 GPU RAM --> CPU RAM
Completed!


.........................


i:1,245,000 5k:250 GPU RAM --> CPU RAM
Completed!


.........

11

In [17]:
cpu_posts = np.concatenate(cpu_posts, axis=0)
cpu_posts.shape

(1292800, 5000)

#### Sample the Data Once Again

In [18]:
# SAMPLE_SIZE = 1000
# id_list = list(range(0, len(matrix_posts)))
# idx = random.sample(id_list, SAMPLE_SIZE)
# sampled_posts = [post for i, post in enumerate(matrix_posts) if i in idx]
# sampled_labels = [label for i, label in enumerate(flat_labels) if i in idx]

# del matrix_posts
# del flat_labels
# len(sampled_posts), len(sampled_labels)

#### Convert from GPU `cupy.ndarray` to main memory `numpy.ndarray`

### Train the Baseline Model

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    cpu_posts,
    labels,
    test_size=TEST_SPLIT,
    random_state=1337
)
# del matrix_posts
# del labels

In [20]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(X_train.dtype, X_test.dtype, y_train.dtype, y_test.dtype)

(1034240, 5000) (258560, 5000) (1034240, 24) (258560, 24)
float32 float32 int64 int64


## Compute Sample Weights

In [21]:
from sklearn.utils.class_weight import compute_sample_weight

train_sample_weights = compute_sample_weight('balanced', y_train).tolist()
test_sample_weights = compute_sample_weight('balanced', y_test).tolist()

train_sample_weights, test_sample_weights

([0.0001318841169496333,
  1.3037444443670205e-07,
  7.764130177354329e-05,
  7.561055902617928e-06,
  0.00011403844703654711,
  4.4243636117116125e-06,
  1.3037444443670205e-07,
  3.458236441128177e-06,
  1.3037444443670205e-07,
  1.3037444443670205e-07,
  1.3037444443670205e-07,
  0.004008547885223326,
  0.0013583527650081573,
  1.3037444443670205e-07,
  1.7905655990743493e-06,
  1.3037444443670205e-07,
  1.3037444443670205e-07,
  1.3037444443670205e-07,
  4.749549830646288e-05,
  1.3037444443670205e-07,
  0.0034087941941906307,
  1.3037444443670205e-07,
  1.3037444443670205e-07,
  0.00011394578379118706,
  1.3037444443670205e-07,
  4.342324842862687e-06,
  1.3037444443670205e-07,
  6.553068698770969e-06,
  0.06785761598738561,
  1.3037444443670205e-07,
  1.3037444443670205e-07,
  2.121211325429623e-06,
  0.00014147245852330704,
  2.9847570194472e-06,
  4.062494283266732e-06,
  5.510694267198438e-06,
  1.3037444443670205e-07,
  1.3037444443670205e-07,
  0.00010491673421662921,
  1.30

## Compute Class Weights

In [22]:
train_weight_vec = list(np.max(np.sum(y_train, axis=0))/np.sum(y_train, axis=0))
train_class_weights = {i: train_weight_vec[i] for i in range(y_train.shape[1])}

test_weight_vec = list(np.max(np.sum(y_test, axis=0))/np.sum(y_test, axis=0))
test_class_weights = {i: test_weight_vec[i] for i in range(y_test.shape[1])}

# sorted(list(train_class_weights.items()), key=lambda x: x[1]), sorted(list(test_class_weights.items()), key=lambda x: x[1])
train_class_weights

{0: 3.064303488016763,
 1: 2.371098500202675,
 2: 2.6609803252587287,
 3: 3.0691268418521274,
 4: 3.4792565055762084,
 5: 1.1721270413786193,
 6: 2.6288903037339426,
 7: 2.1467368034742185,
 8: 2.0044547245780864,
 9: 2.00892933802696,
 10: 1.868153510406132,
 11: 2.328390884665141,
 12: 1.360032550569635,
 13: 2.190823970037453,
 14: 2.2936966964023133,
 15: 2.936618834455926,
 16: 1.0,
 17: 1.621670324592815,
 18: 3.2634711051187875,
 19: 4.003992926815355,
 20: 1.869994938327517,
 21: 2.653536460892904,
 22: 2.1827171242886907,
 23: 2.431382057499134}

In [23]:
# from sklearn.utils import class_weight

# class_weights = class_weight.compute_class_weight(
#     'balanced',
#      np.unique(y_train.tolist()),
#      y_train.tolist()
# )
y_train.shape

(1034240, 24)

In [27]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
    n_estimators=100,
    n_jobs=12,
    class_weight='balanced',
    random_state=1337,
    verbose=1,
)
clf

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=12, oob_score=False,
                       random_state=1337, verbose=1, warm_start=False)

In [ ]:
clf.fit(X_train, y_train, sample_weight=train_sample_weights)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.


In [ ]:
# from scipy import sparse

# X_train = sparse.csr_matrix(X_train)

# params = {
#     'booster': 'gbtree',
#     'silent': 0,
# }

# clf = OneVsRestClassifier(
#     xgb.XGBClassifier(
#         learning_rate=0.2,
#         n_estimators=100,
#         objective='binary:logistic',
#         nthread=1,
#         tree_method='gpu_hist'
#     ), 
#     n_jobs=1,
# )
# %timeit clf.fit(X_train[:200000], y_train[:200000])

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(
#     n_estimators=100,
#     max_depth=3,
#     random_state=33,
#     n_jobs=12
# )
# clf.fit(X_train, y_train)

In [ ]:
# %%bash

# echo 'Fitting done!' | ~/bin/twilio-sms 404-317-3620

In [ ]:
# d_train = xgb.DMatrix(X_train, label=y_train)
# d_test =  xgb.DMatrix(X_test, label=y_test)

In [ ]:
# from sklearn.model_selection import cross_val_score

# cross_val_score(clf, X_train, y_train, cv=2, scoring='accuracy')

In [ ]:
from sklearn.metrics import (
    roc_curve, precision_recall_curve, auc, make_scorer, recall_score, 
    accuracy_score, jaccard_score, precision_score, confusion_matrix
)

tag_labels = [tag[1] for tag in sorted_all_tags]

y_pred = clf.predict(X_test)
# report = classification_report(y_test, y_pred, output_dict=True, target_names=tag_labels)
y_pred.sum(axis=0)

In [ ]:
print(report)

In [ ]:
accuracy_score(y_test, y_pred)

jaccard_score(y_test, y_pred)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, StratifiedKFold

plt.style.use("ggplot")

xgb_params = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}

In [ ]:
from keras import Sequential
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=VOCAB_SIZE, lower=True)
tokenizer.fit_on_texts(post_code)
sequences = tokenizer.texts_to_sequences(post_code)
X = pad_sequences(sequences, maxlen=MAX_LENGTH)
X.shape

model = Sequential()
model.add(Embedding(1000, 64, input_length=MAX_LENGTH))


In [ ]:
model